In [19]:
%reload_ext autoreload
%autoreload 2
from icecream import ic
from pathlib import Path
from models.service_reports.parsers import Enercon_Parser
import fitz
import camelot
import polars as pl

In [45]:
pdf_path = str(Path("data/enercon/input") / "enercon_master_example.pdf")
master_data_params = {
    'flavor': 'stream',
    'columns': ['125, 290, 390'],
    'table_areas': ['20,700,600,620'],
    'row_tol': 13,
    'split_text': True
}

tables = camelot.read_pdf(
    filepath=pdf_path,
    pages='2',
    **master_data_params)

In [46]:
raw_cm_data = tables[0].df

In [48]:
df = pl.from_pandas(raw_cm_data)
df.columns = [i % 2 for i in range(len(df.columns))] 
    # Rename columns to 0,1,0,1 and so on

key_value_pairs = []
for i in range(0, len(df.columns), 2):
    pair = df.iloc[:, i:i+2]
    key_value_pairs.append(pair)
        # stores fragmented dataframe in pairs of value-key columns in 

stacked_df = pl.concat(key_value_pairs, how='vertical')

ValueError: Pandas dataframe contains non-unique indices and/or column names. Polars dataframes require unique string names for columns.

In [43]:
from models.csv_formatter import CSVFormatter
CSVFormatter.stack_column_in_pairs(pl.from_pandas(raw_cm_data), p)

ValueError: Pandas dataframe contains non-unique indices and/or column names. Polars dataframes require unique string names for columns.

In [44]:
raw_cm_data = pl.from_pandas(raw_cm_data)
df = raw_cm_data

ValueError: Pandas dataframe contains non-unique indices and/or column names. Polars dataframes require unique string names for columns.

In [30]:
raw_cm_data.columns = [i % 2 for i in range(len(raw_cm_data.columns))]

In [32]:
raw_cm_data.columns

Index([0, 1, 0, 1], dtype='int64')

In [1]:
from models.service_reports.parsers.enercon_parser2 import Enercon_PDF_Parser
from pathlib import Path

# Initialize parser with example file
pdf_path = Path("data") / "enercon" / "input" / "enercon_master_example.pdf"
parser = Enercon_PDF_Parser(pdf_path)

In [5]:
parser._extract_raw_checklist()
parser._has_report_overview()
parser._set_report_overview()

In [6]:
parser.overview_df

,0,1,2
0,No.,Defect locationDefect type,Report
1,2,"special, control system - Software tower, towe...",920079187006
2,No.,Defect locationDefect type,Report
3,39,"special, electrical component - battery tower,...",920079187005
4,57,"blade system A - cabinet, capacitortesting/saf...",920061438810
5,63,"rotor system - slip ring unitspecial, sealing ...",920079187015
6,96,"tower, tower base - fire extinguishermaterial,...",920074061850
7,96,testing/safety - inspection date missing/tower...,920079187008
8,97,"material, concrete / mortar - crack outside, c...",920079187009
9,97,"material, concrete / mortar - crack inside, co...",920079187007


In [2]:
from models.service_reports.parsers.enercon_parser import Enercon_PDF_Parser
from pathlib import Path

# Initialize parser with example file
pdf_path = Path("data") / "enercon" / "input" / "enercon_master_example.pdf"
parser = Enercon_PDF_Parser(str(pdf_path))

# Get metadata and checklist
metadata_df = parser.get_metadata()
checklist_df = parser.get_inspection_checklist()

# Display results
print("=== METADATA ===")
print(metadata_df)
print("\n=== INSPECTION CHECKLIST ===")
checklist_df.to_csv('test.csv')

=== METADATA ===
                                                          Metadata
Serial number                                                  921
Project number                                             S-07925
Site                                                         Dizy-
WEC                                                           E-92
Control system type                                            CS8
Order type                                      MASTER MAINTENANCE
Order number                                                  9912
Final date of order completion                         Dec 8, 2023
Completion date of report                              Dec 6, 2023
Inspection lot number                                     94000351
Document number                                           D084 6.0
Nominated person in control of the                                
electrical installation during work                               
activities                                   

In [1]:
from models.service_reports.parsers.enercon_parser2 import Enercon_PDF_Parser
import pandas as pd
from pathlib import Path
# Chemin vers un rapport test
test_file = Path('data\enercon\input\enercon_wind_example.pdf')

# Test du parser
parser = Enercon_PDF_Parser(test_file)

# Extraction des métadonnées
print("\n=== Métadonnées ===")
metadata = parser.get_metadata()
print(metadata.head())

# Extraction du checklist
print("\n=== Checklist ===")
checklist = parser.get_inspection_checklist()
print(checklist.head())

# Affichage des dimensions
print("\nDimensions:")
print(f"Metadata: {metadata.shape}")
print(f"Checklist: {checklist.shape}")


=== Métadonnées ===
                     Metadata
Serial number             825
Project number       S-04388-
Site                      Bin
WEC                   E-82 E2
Control system type     CS82a

=== Checklist ===
                               no              check_item result
5                      Order type  WIND BASED MAINTENANCE       
6                    Order number                  940114       
7  Final date of order completion            Feb 25, 2025       
8       Completion date of report            Feb 25, 2025       
9           Inspection lot number                94000430       

Dimensions:
Metadata: (16, 1)
Checklist: (22, 3)


In [1]:
from models.service_reports.processors.enercon_processor import Enercon_Processor
from pathlib import Path

# Setup paths
input_folder = Path("data") / "enercon" / "input"
output_base = Path("test_data") / "enercon" / "output"
metadata_output = output_base / "metadatas"
checklist_output = output_base / "inspection_checklists"

# Initialize and run processor
processor = Enercon_Processor(input_folder=input_folder)
processor(
    metadata_output_folder=metadata_output,
    inspection_checklist_output_folder=checklist_output
)

In [ ]:
from models import  Vestas_Reports_Processor; from pathlib import Path
Vestas_Reports_Processor(
    input_folder=Path("data") / "vestas" / "input"
)(
    metadata_output_folder=Path('data') / 'vestas' / 'output' / 'metadatas',
    inspection_checklist_output_folder=Path('data') / 'vestas' / 'output' / 'inspection_checklists'
)

In [ ]:
from models import  Vestas_Reports_Processor; from pathlib import Path
Vestas_Reports_Processor(
    input_folder=Path("data") / "vestas" / "input"
)(
    metadata_output_folder=Path('data') / 'vestas' / 'output' / 'metadatas',
    inspection_checklist_output_folder=Path('data') / 'vestas' / 'output' / 'inspection_checklists'
)